<div style="width:90%;
            margin:10px;
            padding:8px;
            border:2px solid #FF0000;
            border-radius:20px;
            font-weight:bold;
            font-size:2.5em;
            text-align:center;">
The Space Shuttle Reentry Problem Using JuMP with different solvers
</div>


<div style="width:90%;
            margin:9px;
            font-size:1em;">
            
The aim here is to compare the results of different solvers for the same problem: the space shuttle reentry problem using JuMP. For the integration method, we will use the trapezoidal rule. 

1. [Comparing Linear Solvers](#ls)
2. [Comparing Non Linear Solvers](#nls)
3. [Results](#res)


</div>

### Importing the Model

In [1]:
using JuMP , Ipopt
import HSL_jll
using DataFrames, BenchmarkTools
include("../Problems/JuMP/space_Shuttle_JMP.jl")
include("../Benchmark/utils.jl")
results = DataFrame( :method => String[], 
                    :total_time => String[],
                    :objective_value => Float64[],
                    :iterations => Int64[]);


## 1. Comparing Linear Solvers : MUMPS, HSL_MA57 and HSL_MA27  <a id="ls"></a>

#### 1. MUMPS

In [2]:
model = space_Shuttle_JMP("trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_optimizer_attribute(model,"linear_solver", "mumps")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + MUMPS",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



#### 2. HSL_MA57

In [3]:
import HSL_jll
model = space_Shuttle_JMP("trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma57")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + HSL_MA57",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

#### 3. HSL_MA27

In [4]:
import HSL_jll
model = space_Shuttle_JMP("trapezoidal")
set_silent(model)
set_optimizer(model,Ipopt.Optimizer)
set_attribute(model, "hsllib", HSL_jll.libhsl_path)
set_attribute(model, "linear_solver", "ma27")
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["Ipopt + HSL_MA27",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

## 2. Comparing Nonlinear Solvers : Ipopt, MadNLP and Knitro <a id="nls"></a>

#### 1. Ipopt

The results for the Ipopt solver are found in the first part.

#### 2. MadNLP

In [ ]:
using  MadNLP
model = space_Shuttle_JMP("trapezoidal");
set_silent(model)
set_optimizer(model,MadNLP.Optimizer) #default linear solver : Umfpack
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,["MadNLP + Umfpack",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);
# ~5mn

> the problem for not finding the same solutions with Ipopt + HSL_MA57 and MadNLP comes from the linear solvers used . MadNLP uses Umfpack by default, while we use other linear solvers.

In [6]:
using  MadNLP
using MadNLPHSL
model = space_Shuttle_JMP("trapezoidal");
set_silent(model)
set_optimizer(model,()->MadNLP.Optimizer(linear_solver=Ma57Solver))
b = @benchmark optimize!(model) evals=1 samples=1
println(
    "Final latitude θ = ",
    round(objective_value(model) |> rad2deg;digits=7),
    "°",
)
push!(results,[ "MadNLP + HSL_MA57",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

Final latitude θ = 34.1411659°


#### 3. Knitro

In [7]:
using KNITRO
model = space_Shuttle_JMP("trapezoidal");
set_silent(model)
set_optimizer(model,KNITRO.Optimizer);
set_attribute(model, "algorithm", 4);
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,[ "KNITRO_SQP",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

In [ ]:
using KNITRO
model = space_Shuttle_JMP("trapezoidal");
set_silent(model)
set_optimizer(model,KNITRO.Optimizer);
set_attribute(model, "algorithm", 1);
b = @benchmark optimize!(model) evals=1 samples=1
push!(results,[ "KNITRO_IPM",
                prettytime(b.times[1]),
                round(objective_value(model) |> rad2deg;digits=7),
                solution_summary(model).barrier_iterations
                ]);

> #### Knitro does not solve the problem: more than 1000 iterations and 15 minutes of execution time.

## 1. Results <a id="res"></a>

In [23]:
using PrettyTables
using Colors
# Define the custom display function
hl_v1 = Highlighter(
           (results, i, j) -> (j == 3) && (round(results[i, j],digits=4) != 34.1412),
           crayon"red bold"
       );
hl_v2 = Highlighter(
            (results, i, j) -> (j == 3) && (results[i, j] != 34.1411822),
            crayon"red"
        );
header = ["Method", "Total Time", "Objective Value", "Iterations"];
# Apply the custom display function to the :objective_value column
pretty_table(
    results;
    formatters    = ft_printf("%.7f", 2:3),
    header        = header,
    header_crayon = crayon"yellow bold",
    highlighters  = (hl_v1, hl_v2),
    tf            = tf_unicode_rounded
)

╭───────────────────┬───────────────┬─────────────────┬────────────╮
│            Method │    Total Time │ Objective Value │ Iterations │
├───────────────────┼───────────────┼─────────────────┼────────────┤
│     Ipopt + MUMPS │  13.5869269 s │      34.1411822 │        124 │
│  Ipopt + HSL_MA57 │   9.2184755 s │      34.1411822 │        124 │
│  Ipopt + HSL_MA27 │   8.0530951 s │      34.1411822 │        124 │
│  MadNLP + Umfpack │ 118.9959937 s │      34.1161134 │       1732 │
│ MadNLP + HSL_MA57 │  12.4442198 s │      34.1411659 │        210 │
╰───────────────────┴───────────────┴─────────────────┴────────────╯
